In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logregdoc').getOrCreate()

import warnings
warnings.filterwarnings("ignore")

In [8]:
from pyspark.ml.classification import LogisticRegression

In [16]:
# Load training data
data = spark.read.csv('titanic.csv', header=True, inferSchema=True)

In [17]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [21]:
data.createTempView('df')

In [22]:
resutls = spark.sql("SELECT * from df")

In [23]:
resutls.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [25]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [26]:
my_cols = data.select('Survived','Pclass','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked')

In [29]:
my_final_data = my_cols.na.drop()

In [30]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, StringIndexer, OneHotEncoder)

In [31]:
genderIndexer = StringIndexer(inputCol='Sex', outputCol='SexIndexer')
gender_encoder = OneHotEncoder(inputCol='SexIndexer',outputCol='SexVec')

In [42]:
embarkIndexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndexer')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndexer',outputCol='EmbarkVec')

In [43]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec','EmbarkVec','Age','SibSp','Parch','Fare'],
                outputCol = 'features')

In [44]:
from pyspark.ml import Pipeline

In [45]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [46]:
pipeline = Pipeline(stages = [genderIndexer, embarkIndexer, gender_encoder, embark_encoder, assembler, log_reg_titanic])

In [47]:
train_data, test_data = my_final_data.randomSplit([0.7, 0.3])

In [48]:
fit_model = pipeline.fit(train_data)

In [49]:
results = fit_model.transform(test_data)

In [54]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
+--------+----------+
only showing top 20 rows



In [51]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [55]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [56]:
AUC = my_eval.evaluate(results)

In [57]:
AUC

0.663888888888889